<a href="https://colab.research.google.com/github/leomensah/USING-EFFICIENTNET-FOR-LUNG-CANCER-DETECTION/blob/main/EFFICIENT_NET_SEVEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install albumentations==0.4.6

In [ ]:
import os
import numpy as np
import glob
from PIL import Image
import argparse
import os

import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms.functional as TF
import torchvision
from torchvision import transforms

import albumentations as albu
from albumentations.pytorch import ToTensorV2
from albumentations import OneOf,Compose

class ClassifierDataset(Dataset):
    def __init__(self, IMAGES_PATHS,label,Albumentation=False):
      self.image_paths = IMAGES_PATHS
      self.labels = label
      self.albumentation = Albumentation

      self.albu_transformations =  albu.Compose([
          albu.Normalize(),
          ToTensorV2(),
          OneOf([albu.HorizontalFlip(),
                  albu.VerticalFlip(),
                  albu.RandomRotate90(),
                  ],p=0.9)
          #albu.ElasticTransform(alpha=1.1,alpha_affine=0.5,sigma=5,p=0.15),
      ])
      self.transformations = transforms.Compose([
          #transforms.Grayscale(3),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def transform(self, image):
      #Transform to tensor
      if self.albumentation:
          augmented=  self.albu_transformations(image=image)
          image = augmented['image']
      else:
          image = self.transformations(image)
      image= image.type(torch.FloatTensor),
      return image

    def __getitem__(self, index):
      image = np.load(self.image_paths[index])
      image = self.transform(image)
      return image[0],self.labels[index]

    def __len__(self):
      return len(self.image_paths)

def str2bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def load_directories(root_dir):
    images_list = os.listdir(root_dir)
    images_list.sort()
    return [root_dir+ x for x in images_list]

import numpy as np
import torch
import torch.nn.functional as F

def Accuracy(output, label):
  total = 0
  correct =0
  output_prob = (output>0.5).int()
  correct += torch.sum(output_prob==label).item()
  total += len(label)

  return  correct / total

def Confusion_matrix(output, label):
  total = 0
  smooth = 0.1

  output_prob = (output>0.5).int()
  label = label.int()

  conf_matrix = torch.zeros(2, 2)
  for t, p in zip(label, output_prob):
      conf_matrix[t, p] += 1
  TP = conf_matrix[1,1].item()
  TN = conf_matrix[0,0].item()
  FP = conf_matrix[0,1].item()
  FN = conf_matrix[1,0].item()

  total += len(label)
  accuracy = (TP+TN)/total
  sensitivity = TP / (TP+FN+ smooth)
  specificity = TN / (TN+FP+ smooth)
  
  return  accuracy, sensitivity, specificity

def Confusion_matrix2(output, label):
  total = 0
  smooth = 0.1

  output_prob = (output>0.5).int()
  label = label.int()

  conf_matrix = torch.zeros(2, 2)
  for t, p in zip(label, output_prob):
      conf_matrix[t, p] += 1
  TP = conf_matrix[1,1].item()
  TN = conf_matrix[0,0].item()
  FP = conf_matrix[0,1].item()
  FN = conf_matrix[1,0].item()

  return  TP,TN,FN,FP

## **TRAINING THE CLASSIFIER**

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=7c281a71ac1e30af85437b6955f068681f19c4d2635fa859283d52ccf0a31675
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
import pandas as pd
import numpy as np
import argparse
import os
from collections import OrderedDict
from glob import glob
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import yaml

import torch
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from torchvision import datasets, models, transforms
from PIL import Image
from efficientnet_pytorch import EfficientNet

def train(train_loader,model,criterion,optimizer):
    avg_meters = {'loss': AverageMeter(),
                'accuracy': AverageMeter(),
                'sensitivity':AverageMeter(),
                'specificity': AverageMeter()}

    model.train()
    pbar = tqdm(total=len(train_loader))
    for images, labels in train_loader:
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)
        outputs = outputs.view(-1)
        labels = labels.type_as(outputs)
        loss = criterion(outputs, labels)
        accuracy, sensitivity, specificity = Confusion_matrix(outputs,labels)
        #print(loss)

        # compute gradient and do optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meters['loss'].update(loss.item(),images.size(0))
        avg_meters['accuracy'].update(accuracy,images.size(0))
        avg_meters['sensitivity'].update(sensitivity,images.size(0))
        avg_meters['specificity'].update(specificity,images.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('accuracy', avg_meters['accuracy'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg),
            ('specificity', avg_meters['specificity'].avg)
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('accuracy', avg_meters['accuracy'].avg),
                        ('sensitivity', avg_meters['sensitivity'].avg),
                        ('specificity', avg_meters['specificity'].avg),])
    
def validate(val_loader,model,criterion):
    avg_meters = {'loss': AverageMeter(),
                'accuracy': AverageMeter(),
                'sensitivity':AverageMeter(),
                'specificity': AverageMeter()}
    model.eval()

    pbar = tqdm(total=len(val_loader))
    for images, labels in val_loader:
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)
        outputs = outputs.view(-1)
        labels = labels.type_as(outputs)
        loss = criterion(outputs, labels)
        accuracy, sensitivity, specificity = Confusion_matrix(outputs,labels)

        avg_meters['loss'].update(loss.item(),images.size(0))
        avg_meters['accuracy'].update(accuracy,images.size(0))
        avg_meters['sensitivity'].update(sensitivity,images.size(0))
        avg_meters['specificity'].update(specificity,images.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('accuracy', avg_meters['accuracy'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg),
            ('specificity', avg_meters['specificity'].avg)
        ])

        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('accuracy', avg_meters['accuracy'].avg),
                        ('sensitivity', avg_meters['sensitivity'].avg),
                        ('specificity', avg_meters['specificity'].avg),])

In [ ]:
# Model Output directory
OUTPUT_DIR = '/content/drive/MyDrive/data/Classification/model_output/'
filename = 7
epochs = 200
batch_size = 24
early_stopping = 30
num_workers = 2
optimizer = 'Adam'
learning_rate = 1e-6
momentum = 0.9
weight_decay = 1e-4


os.makedirs(OUTPUT_DIR+'efficientnetb{}'.format(filename),exist_ok=True)
print('Made directory called efficientnetb{}'.format(filename))

print('-' * 20)

# Data directory
TRAIN_DIR = '/content/drive/MyDrive/data/Efficient_net/train/'
VAL_DIR = '/content/drive/MyDrive/data/Efficient_net/val/'
CLEAN_TRAIN_DIR ='/content/drive/MyDrive/data/Efficient_net/clean_train/'
CLEAN_VAL_DIR = '/content/drive/MyDrive/data/Efficient_net/clean_val/'
LABEL_DIR = '/content/drive/MyDrive/data/Efficient_net/label/'

with open(LABEL_DIR+'train.txt','rb') as fp:
  train_label = pickle.load(fp)
with open(LABEL_DIR+'val.txt','rb') as fp:
  val_label = pickle.load(fp)
with open(LABEL_DIR+'clean_train.txt','rb') as fp:
  clean_train_label = pickle.load(fp)
with open(LABEL_DIR+'clean_val.txt','rb') as fp:
  clean_val_label = pickle.load(fp)

# Get image files path as list
train_image_paths = load_directories(TRAIN_DIR)
val_image_paths = load_directories(VAL_DIR)
clean_train_images_paths = load_directories(CLEAN_TRAIN_DIR)
clean_val_images_paths = load_directories(CLEAN_VAL_DIR)

train_image_paths.extend(clean_train_images_paths)
val_image_paths.extend(clean_val_images_paths)
train_label.extend(clean_train_label)
val_label.extend(clean_val_label)

print("="*50)
print("The length of image are train: {} validation: {}".format(len(train_image_paths),len(val_image_paths)))

print("============================TRAINING===========================================")
print("Cancer nodules:{} Non Cancer nodules:{}".format(np.sum(train_label),len(train_label)-np.sum(train_label)))
print("Ratio is {:4f}".format(np.sum(train_label)/(len(train_label)-np.sum(train_label))))
print("============================VALIDATION=========================================")
print("Cancer nodules:{} Non Cancer nodules:{}".format(np.sum(val_label),len(val_label)-np.sum(val_label)))
print("Ratio is {:4f}".format(np.sum(val_label)/(len(val_label)-np.sum(val_label))))
# Create Dataset
train_dataset = ClassifierDataset(train_image_paths,train_label)
val_dataset = ClassifierDataset(val_image_paths,val_label)

# Model
cudnn.benchmark = True
model = EfficientNet.from_pretrained('efficientnet-b{}'.format(filename))

#Fine tuning top layers
num_ftrs = model._fc.in_features
model._fc = nn.Sequential(nn.Linear(num_ftrs,1),
                            nn.Sigmoid())

criterion = nn.BCEWithLogitsLoss().cuda()


if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model = model.cuda()
params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(params, lr=learning_rate, weight_decay=weight_decay)

#if config['optimizer'] == 'Adam':
#elif config['optimizer'] == 'SGD':
#    optimizer = optim.SGD(params, lr=config['lr'], momentum=config['momentum'],nesterov=config['nesterov'], weight_decay=config['weight_decay'])
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
# Create Dataloader
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=12,
    shuffle=True,
    pin_memory=True,
    drop_last=True,
    num_workers=6)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=12,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=6)

log= pd.DataFrame(index=[],columns= ['epoch', 'loss', 'accuracy','sensitivity','specificity,',
                                    'val_loss', 'val_accuracy','val_sensitivity','val_specificity'])

best_loss= 10
trigger = 0

for epoch in range(epochs):

    # train for one epoch
    train_log = train(train_loader, model, criterion, optimizer)
    # evaluate on validation set
    val_log = validate(val_loader, model, criterion)

    print('Training epoch [{}/{}], Training BCE loss:{:.4f}, Training accuracy:{:.4f}, Training sensitivity:{:.4f}, Training specificity:{:.4f}, \
                Validation BCE loss:{:.4f}, Validation accuracy:{:.4f}, Validation sensitivity:{:.4f}, Validation specificity:{:.4f},'.format( 
        epoch + 1, epochs, train_log['loss'], train_log['accuracy'], train_log['sensitivity'], train_log['specificity'],val_log['loss'], val_log['accuracy'],val_log['sensitivity'], val_log['specificity']))

    tmp = pd.Series([
        epoch,
        train_log['loss'],
        train_log['accuracy'],
        train_log['sensitivity'],
        train_log['specificity'],
        val_log['loss'],
        val_log['accuracy'],
        val_log['sensitivity'],
        val_log['specificity'],
    ], index=['epoch', 'loss', 'accuracy','sensitivity','specificity,','val_loss', 'val_accuracy','val_sensitivity','val_specificity'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv(OUTPUT_DIR+'efficientnetb{}/log.csv'.format(filename), index=False)

    trigger += 1

    if val_log['loss'] < best_loss:
        torch.save(model.state_dict(), OUTPUT_DIR+'efficientnetb{}/model.pth'.format(filename))
        best_loss= val_log['loss']
        print("=> saved best model as validation loss is greater than previous best loss")
        trigger = 0

    # early stopping
    if early_stopping >= 0 and trigger >= early_stopping:
        print("=> early stopping")
        break

    torch.cuda.empty_cache()

Made directory called efficientnetb7
--------------------
The length of image are train: 5920 validation: 2042
============================TRAINING===========================================
Cancer nodules:3146 Non Cancer nodules:2774
Ratio is 1.134102
============================VALIDATION=========================================
Cancer nodules:914 Non Cancer nodules:1128
Ratio is 0.810284
Loaded pretrained weights for efficientnet-b7


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 171/171 [02:01<00:00,  1.41it/s, loss=0.747, accuracy=0.555, sensitivity=0.316, specificity=0.393]


Training epoch [1/200], Training BCE loss:0.7056, Training accuracy:0.5565, Training sensitivity:0.5676, Training specificity:0.5363,                 Validation BCE loss:0.7471, Validation accuracy:0.5548, Validation sensitivity:0.3164, Validation specificity:0.3934,
=> saved best model as validation loss is greater than previous best loss


100%|██████████| 171/171 [00:18<00:00,  9.17it/s, loss=0.741, accuracy=0.614, sensitivity=0.324, specificity=0.454]


Training epoch [2/200], Training BCE loss:0.6998, Training accuracy:0.6190, Training sensitivity:0.6225, Training specificity:0.6126,                 Validation BCE loss:0.7413, Validation accuracy:0.6141, Validation sensitivity:0.3243, Validation specificity:0.4544,
=> saved best model as validation loss is greater than previous best loss


...

Training epoch [57/200], Training BCE loss:0.5074, Training accuracy:0.9708, Training sensitivity:0.9537, Training specificity:0.9621,                 Validation BCE loss:0.6595, Validation accuracy:0.7023, Validation sensitivity:0.3376, Validation specificity:0.5710,


100%|██████████| 171/171 [00:18<00:00,  9.23it/s, loss=0.661, accuracy=0.706, sensitivity=0.346, specificity=0.568]


Training epoch [58/200], Training BCE loss:0.5051, Training accuracy:0.9763, Training sensitivity:0.9593, Training specificity:0.9701,                 Validation BCE loss:0.6614, Validation accuracy:0.7057, Validation sensitivity:0.3464, Validation specificity:0.5683,


100%|██████████| 171/171 [00:18<00:00,  9.25it/s, loss=0.662, accuracy=0.706, sensitivity=0.345, specificity=0.57]

Training epoch [59/200], Training BCE loss:0.5044, Training accuracy:0.9777, Training sensitivity:0.9608, Training specificity:0.9683,                 Validation BCE loss:0.6620, Validation accuracy:0.7062, Validation sensitivity:0.3447, Validation specificity:0.5698,
=> early stopping


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   13340 MB |   13449 MB |  109345 MB |   96005 MB |\n|       from large pool |   13114 MB |   13380 MB |  108861 MB |   95747 MB |\n|       from small pool |     225 MB |     225 MB |     483 MB |     258 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   13340 MB |   13449 MB |  109345 MB |   96005 MB |\n|       from large pool |   13114 MB |   13380 MB |

## **EVALUATE THE EFFICENT NETWORK MODEL**

In [ ]:
def evaluate(model,test_loader):
    with torch.no_grad():
        counter = 0
        pbar = tqdm(total=len(test_loader))
        TP,TN,FN,FP = 0,0,0,0
        for images, labels in test_loader:
            images = images.cuda()
            labels = labels.cuda()

            output = model(images)
            outputs = output.view(-1)
            labels = labels.type_as(outputs)
            TP_batch,TN_batch,FN_batch,FP_batch = Confusion_matrix2(outputs,labels)
            TP += TP_batch
            TN += TN_batch
            FN += FN_batch
            FP += FP_batch
            
            pbar.update(1)
        pbar.close()

        accuracy = (TP+TN)/ (TP +TN +FN +FP)
        sensitivity = TP / (TP+FN)
        specificity = TN / (TN+FP)
        precision = TP / (TP+FP)
        F1 = (2*sensitivity*precision)/(precision+sensitivity)
    torch.cuda.empty_cache()
    return accuracy, sensitivity, specificity, precision,F1, TP,TN,FN,FP

In [ ]:
filename = 7
epochs = 200
batch_size = 12
early_stopping = 30
num_workers = 2
optimizer = 'Adam'
learning_rate = 1e-6
momentum = 0.9
weight_decay = 1e-4

OUTPUT_DIR = '/content/drive/MyDrive/data/Classification/model_output/efficientnetb{}/'.format(filename)
TEST_DIR = '/content/drive/MyDrive/data/Efficient_net/test/'
CLEAN_TEST_DIR = '/content/drive/MyDrive/data/Efficient_net/clean_test/'
LABEL_DIR ='/content/drive/MyDrive/data/Efficient_net/label/'

with open(LABEL_DIR+'test.txt','rb') as fp:
    test_label = pickle.load(fp)
with open(LABEL_DIR+'clean_test.txt','rb') as fp:
    clean_test_label = pickle.load(fp)

print('-'*20)

cudnn.benchmark = True
model = EfficientNet.from_pretrained('efficientnet-b{}'.format(filename))

#Fine tuning top layers
num_ftrs = model._fc.in_features
model._fc = nn.Sequential(nn.Linear(num_ftrs,1),
                            nn.Sigmoid())

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.load_state_dict(torch.load(OUTPUT_DIR +'model.pth'))
model = model.cuda()

# Get image files
test_image_paths = load_directories(TEST_DIR)
clean_test_image_paths = load_directories(CLEAN_TEST_DIR)

test_image_paths.extend(clean_test_image_paths)
test_label.extend(clean_test_label)

assert len(test_image_paths)==len(test_label), "Length of test images and test label not same"
print("============================TESTING===========================================")
print("Cancer nodules:{} Non Cancer nodules:{}".format(np.sum(test_label),len(test_label)-np.sum(test_label)))
print("Ratio is {:4f}".format(np.sum(test_label)/(len(test_label)-np.sum(test_label))))

test_dataset = ClassifierDataset(test_image_paths,test_label,Albumentation=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=6)

log= pd.DataFrame(index=[],columns= ['test_size','accuracy','sensitivity','specificity','TP','TN','FN','FP'])

accuracy, sensitivity, specificity, precision,F1, TP,TN,FN,FP = evaluate(model,test_loader)

tmp = pd.Series([
    len(test_dataset),accuracy, sensitivity, specificity, TP, TN, FN, FP
], index=['test_size','accuracy','sensitivity','specificity','TP','TN','FN','FP'])

print('Test accuracy:{:.4f}, Test sensitivity:{:.4f}, Testspecificity:{:.4f}'.format(accuracy,sensitivity,specificity))
print('Test precision:{:.4f}, Test F1:{:.4f}'.format(precision,F1))
log = log.append(tmp,ignore_index=True)
log.to_csv(OUTPUT_DIR +'test_result.csv',index=False)
print("OUTPUT RESULT SAVED AS CSV in", OUTPUT_DIR)

--------------------


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


============================TESTING===========================================
Cancer nodules:1189 Non Cancer nodules:972
Ratio is 1.223251


100%|██████████| 181/181 [01:37<00:00,  1.86it/s]

Test accuracy:0.5664, Test sensitivity:0.5257, Testspecificity:0.6163
Test precision:0.6263, Test F1:0.5716
OUTPUT RESULT SAVED AS CSV in /content/drive/MyDrive/data/Classification/model_output/efficientnetb7/


Certain Methods to employ for optimization.

1. Change the Optimizer from Adam to SGD
2. Quantization
3. Augmentation
4. AutoML
5. Use Different Pretrained models
6. Retrain the model weights with the dataset